## Key Concepts ##

RKnot builds a simulation across four dimensions of global properties:

* **Time**
    * The fundamental unit of time is a `tick`. 
    * Each iteration of the simulation is one tick. During a tick, the following occurs:
        * subjects can move to new locations
        * subjects can have contact with other subjects
        * attributes of the subjects can change
    * in the real world, many of the fundamental properties of a virus are measured in days (i.e. recovery rate or duration of immunity). RKnot translates these inputs into ticks.
    * Currently, RKnot only supports one `tick` per day. The goal is to support any number of ticks during a day.
* **Space**
    * Subjects interact in an two-dimensional environment called the Grid.
    * The Grid *must* be a square. The Grid size can be passed manually or it can be determined automatically for a specified density level.
    * Each pair of xy coordinates in the Grid is a location. 
    * A *contact* occurs when an infected subject and a susceptible subject occupy the same location on the same tick.
    * A single subject can only occupy a single location at one tick. There is no limit to the number of subjects that can occupy a single location at one tick.
    * Subjects move through the Grid according to user-specified `mover` [functions](#Mover-Functions). These functions typically incorporate a degree of randomness.
    * A subject can also move by attending an [Event](#Events).
    * Portions of the Grid may be restricted by [Boxes and/or Gates](#Boxes-and-Gates).

* **Subjects**
    * subjects (also referred to as "dots") are the analog of people in the simulation.
    * subjects carry many attributes through the life of the simulation that are updated and changed as required [see Dot Matrix](#Dot-Matrix)).
* **Virus**
    * the user must pass several characteristics fundamental to the simulated virus. RKnot infers others. Virus characteristics include:
        * $R_0$
        * Duration of Infection
        * Transmission Risk        
        * Duration of Immunity
        * Infection Fatality Rate

### The Sim ###
The `Sim` object is the user interface for the RKnot simulation package. A Sim object is instantiated with pre-defined characteristics of the space, the subjects, and the virus.

For demonstration purposes, a quick default simulation can be run by simply providing a few parameters.

In [1]:
### HIDDEN CELL ###
%load_ext autoreload
%autoreload 2

from IPython.core.display import HTML

RUN = False
SHOW_CHARTS = False
SAVE_CHARTS = False

PATH = 'vids/concepts/tests/'

```python
from rknot import Sim, Chart

params = {'square': 4, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
group = {'n': 2, 'n_inf': 1}
sim = Sim(groups=group, **params)
```

`run` is the main method of the `Sim` object. `run` iterates through each `tick` in the simulation. Currently, one day == one tick.

```python
sim.run()
```

`sim.run()` does not return any values, but it does update various attributes of the `sim` object. After calling `run`, you can then pass `sim` to the `Chart` object, which will generate an animation of the simulation across time.

```python
chart = Chart(sim, dotsize=2000, interval=200, show_intro=False, use_init_func=False)
chart.to_html5_video()
```

In [2]:
if RUN:
    from rknot import Sim, Chart
    params = {'square': 4, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
    sim = Sim(**params)
    sim.run()

if SHOW_CHARTS:
    chart = Chart(
        sim, dotsize=2000, show_intro=False, 
        use_init_func=False, interval=200,
    )
    HTML(chart.to_html5_video())

if SAVE_CHARTS:
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        dotsize=200, interval=200, reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'intro.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/intro.mp4' controls>alternative text</video>
<p></p>

The animation is split in 3 sections: 
* **Interactions**
    * the visual representation of subjects in the Grid. Each marker is a subject and each cross-section of gridlines is a point (for larger grids the lines are removed).
* **Details**
    * provides several on-the-run statistics including Effective Reproduction Number, total fatalities, and fatalities by group.
* **Infections**
    * shows the change in infection level over each day, showing both current infection level and total penetration ("Ever" in the legend)
    
The animation is built on components that can be viewed in various preset layouturations. [see Chart for more details](#Chart).

As per the animation above, the default simulation is of a single infected subject, moving across a 4x4 two-dimensional space according to the `equal` mover function. The subject is equally likely to move to any location in the Grid on any tick.

___
### Subjects ###

#### Dots ###
Dots are subjects/people in the simulation space. A subject has many attributes that are adjusted over time, including:

* which Group it belongs to
* if it is alive
* if it is infected
* if it is susceptible
* its location
* any restricted areas that apply to it ([see Boxes and Gates](#Boxes-and-Gates))
* if infected, when it will recover (or when it will succumb)
* if recovered, when it will again become susceptible
* its fatality rate
* its `mover` function

[see Dot Matrix](#Dot-Matrix) for a more fulsome discussion.

#### Groups ####
Dots are the fundamental subjects of the simulation, but dots can **only** be created via a Group object.

To create our group objects, we can pass a list of dictionaries to the `groups` parameter of Sim. The dictionaries correspond to the attributes of the group, which in turn correspond to the attributes of its constituent dots at initiation.

To create a group, you need only provide two parameters:

* `n`, population size of the group at initiation
* `n_inf`, number of infected subjects in the group at initiation

*If only one group is being provided, you can pass a dictionary. With multiple groups, pass an iterable of dicts.*

```python
params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
group = dict(n=2, n_inf=1)

sim = Sim(groups=group, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, interval=200,
    layout='dots_only', show_intro=False
)
chart.animate.to_html5_video()
```

Below we see this structure creates a 10x10 Grid with two subjects, only one of which is infected at the outset.

In [3]:
if RUN:
    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
    group = dict(n=2, n_inf=1)

    sim = Sim(groups=group, **params)
    sim.run()
if SHOW_CHARTS:
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        dotsize=200, interval=200, 
        layout='dots_only', reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'group1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/group1.mp4' controls>alternative text</video>
<p></p>

There are many other parameters and customizations that can provided:

* `name`
    * if not provided, Sim will create one
* `box` & `box_is_gate` 
    * see [Boxes and Gates](#Boxes-and-Gates)
* `mover`
    * function used to dictate a dot's movement ([see Mover Functions](#Mover-Functions))
* `tmf`
    * [*transmission factor*](theory.ipynb), $T$, applied to each of the dots interactions.
    * default: 1 
* `susf`
    * *susceptiblity factor*; the fraction of subjects in a group that will be made susceptible to the virus at initiation
    * the inverse of `susf` ($1/{susf}$) is the number of subjects in a group that already have immunity.
* `ifr`
    * *infection fatality rate*; or the likelihood that an infection will be fatal

These can again be passed as a dictionary of a single group: 

```python
group = dict(
    name='main', n=2, n_inf=1, mover='equal', 
    tmf=1.25, susf=.75, ifr=0.005
)

sim = Sim(groups=group, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, interval=200, 
    layout='dots_only', show_intro=False
) 
chart.to_html5_video()
```

In [4]:
if RUN:
    group = dict(name='main', n=2, n_inf=1, mover='local', tmf=1.25, susf=.75, ifr=0.005)

    sim = Sim(groups=group, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, figsize=(16,8), dotsize=2000, interval=200, 
        layout='dots_only', show_intro=False
    ) 
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        dotsize=200, interval=200, 
        layout='dots_only', reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'group2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/group2.mp4' controls>alternative text</video>
<p></p>

Or as an iterable of dictionaries. Each group is assigned a unique marker in the animation.

```python
group1 = dict(name='1', n=1, n_inf=1, mover='local', tmf=1.25, susf=.75, ifr=0.005)
group2 = dict(name='2', n=1, n_inf=0, mover='equal', tmf=0.75, susf=0.95, ifr=0.05)
group3 = dict(name='3', n=1, n_inf=0, mover='equal', tmf=0.25, susf=0.5, ifr=0.4)    
groups = [group1, group2, group3]

sim = Sim(groups=groups, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, interval=200, 
    layout='dots_only', show_intro=False
)
chart.to_html5_video()
```

In [5]:
if RUN:
    from rknot import Sim
    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
    
    group1 = dict(name='1', n=1, n_inf=1, mover='local', tmf=1.25, susf=.75, ifr=0.005)
    group2 = dict(name='2', n=1, n_inf=0, mover='equal', tmf=0.75, susf=0.95, ifr=0.05)
    group3 = dict(name='3', n=1, n_inf=0, mover='equal', tmf=0.25, susf=0.5, ifr=0.4)    
    groups = [group1, group2, group3]

    sim = Sim(groups=groups, **params)
    sim.run()

if SHOW_CHARTS:
    from rknot import Chart
    chart = Chart(
        sim, figsize=(16,8), dotsize=2000, interval=200, 
        layout='dots_only', show_intro=False
    )

    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    from rknot import Chart    
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        dotsize=200, interval=200, 
        layout='dots_only', reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'group3.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/group3.mp4' controls>alternative text</video>
<br>

___
### The Grid ###

All interactions in an RKnot simulation take place inside the Grid. The grid is a `Grid` object, which in turn is a sub-classed numpy array with some additional features.

The Grid size can be determined by passing the `square` parameter or the `dlevel` parameter. Each `dlevel` corresponds to a certain density.

In [6]:
from rknot.helpers import md
from rknot import Sim
msg = 'Available `dlevel` options and their corresponding densities are: \n\n'
dlevels = ['{}: {}\n\n'.format(level, density) for level, density in Sim.DENSITIES.items()]
msg += ''.join(dlevels)
md(msg)

Available `dlevel` options and their corresponding densities are: 

low: 0.2

med: 1

high: 10



If we set `dlevel=med`, the Grid will be set such that the density is 1 subject per location. For a group of 100 subjects, that will result in a 10x10 grid. We can see these attributes by passing `details=True`.

In [7]:
params = {'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
group = dict(name='main', n=100, n_inf=1)

sim = Sim(groups=group, dlevel='med', details=True, **params)

---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           Boundary|      [ 1 10  1 10]|          Locations|                100|
|-------------------|-------------------|-------------------|-------------------|
|         Population|                100|            Density|                1.0|
|-------------------|-------------------|-------------------|-------------------|
|       Contact Rate|               1.01|                   |                   |
|-------------------|-------------------|-------------------|-------------------|


```python
sim.run()

chart = Chart(
    sim, figsize=(16,8), layout='dots_only', 
    show_intro=False, use_init_func=False
)
chart.to_html5_video()
```

In [8]:
if RUN:
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)
    HTML(chart.to_html5_video())

if SAVE_CHARTS:
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        interval=200, 
        layout='dots_only', reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'grid1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/grid1.mp4' controls>alternative text</video>
<p></p>

For smaller populations, the density level can only be approximated. RKnot defaults to rounding up to the nearest square value.

You can also pass a float value to `dlevel` in order to create a specified density. Here, we set `dlevel=3.5`

In [9]:
group1 = dict(name='1', n=1000, n_inf=1)
group2 = dict(name='2', n=20, n_inf=20)
groups = [group1, group2]

sim = Sim(groups=groups, dlevel=3.5, details=True, **params)

---------------------------------------------------------------------------------
|                                  SIM DETAILS                                  |
|-------------------------------------------------------------------------------|
|           Boundary|      [ 1 18  1 18]|          Locations|                324|
|-------------------|-------------------|-------------------|-------------------|
|         Population|              1,020|            Density|               3.15|
|-------------------|-------------------|-------------------|-------------------|
|       Contact Rate|               3.16|                   |                   |
|-------------------|-------------------|-------------------|-------------------|


```python
sim.run()

chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)
chart.to_html5_video()
```

In [10]:
if RUN:
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
        show_intro=False, use_init_func=False, 
        interval=200, 
        layout='dots_only', reso_format='web'
    )
    chart = Chart(sim, **chart_params).save(PATH + 'grid2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/grid2.mp4' controls>alternative text</video>
<br>

___
#### Mover Functions ####

When a subject changes locations, this is called a ‘move’. A move is completed during a tick and the movement of a subject on any tick is governed by its mover function. Movers select a location according to a pre-defined probability distribution, so the general movement pattern of a dot can be pre-determined, but any one movement occurs randomly.

There are currently 5 mover functions. Their respective definitions, along with examples of their movement are provided below.

In [11]:
### HIDDEN ###
from rknot.space import MOVERS

if SAVE_CHARTS:
    for mover in MOVERS:
        group1 = dict(name='1', n=1, n_inf=1, mover=mover.__name__)

        sim = Sim(groups=group1, square=10, **params)
        sim.run()

        chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, 
            layout='dots_only', reso_format='web'
        )
        chart = Chart(sim, **chart_params).save(PATH + '{}.mp4'.format(mover.__name__))

In [22]:
from rknot.space.movers import MOVER_DEFNS, MOVER_NAMES
text = ''
for i, mover in enumerate(MOVER_NAMES):
    text += '\n\n{}: '.format(mover.capitalize())
    text += '<p>{}</p>'.format(MOVER_DEFNS[i])
    text += '<video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/{}.mp4" controls>alternative text</video>'.format(mover)
    text += '<p></p>'

md(text)



Equal: <p>The subject is equally likely to move to any location.</p><video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/equal.mp4" controls>alternative text</video><p></p>

Local: <p>The subject has a strong bias towards dots only in its immediate vicinity.</p><video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/local.mp4" controls>alternative text</video><p></p>

Traveller: <p>The subject commonly moves to locations far across the Grid.</p><video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/traveller.mp4" controls>alternative text</video><p></p>

Quarantine: <p>The subject has a strong bias towards not moving, with only some movement occuring.</p><video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/quarantine.mp4" controls>alternative text</video><p></p>

Social: <p>The subject moves mostly within its vicinty but also to other more medium distance location.</p><video width="705" height="340" src="https://storage.googleapis.com/rknotvids/keycons/social.mp4" controls>alternative text</video><p></p>

Movement can also be governed by [Boxes and Gates](#Boxes-and-Gates) or [Events](#Events).
 

___
### Boxes and Gates ###
The movement of a subject across the Grid can be restricted by two concepts known as Boxes and Gates. These concepts are designed to mimick certain funcitonal or perceived boundaries between groups, such as international borders or closed-access communities like assisted-living facilities.

The distinction between boxes and gates is simple:

* Subjects *cannot* **exit** Boxes
* Subjects *cannot* **enter** Gates

___
#### Boxes ####
A box is a $m*n$ subset of locations within the Grid that a subject(s) cannot leave.

The locations are specified by passing a four element iterable that specifies the coordinates of the "four corners" of the box according to [$x_0$, $x_1$, $y_0$, $y_1$]

So passing:
```python
box = [2,6,3,9]
```
creates a box with the four corners:
```python
(2,3)   (2,9)    (6,3)   (6,9)
```

In [13]:
from rknot.space import Box
box = Box([2,6,3,9])
msg = "and a total of {} locations.".format(box.n_locs)

md(msg)

and a total of 35 locations.

Currently, a box *can only be specified* by passing the `box` parameter as group keyword. Every dot in the group is can only move within the `box`, regardless of the size of the Grid.

```python
group1 = dict(name='1', n=2, n_inf=1, box=[1,3,2,4])

sim = Sim(groups=group1, square=10, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, 
    layout='dots_only', show_intro=False, use_init_func=False
)
chart.to_html5_video()
```

In [14]:
if RUN:
    group1 = dict(name='1', n=2, n_inf=1, box=[1,3,2,4])

    sim = Sim(groups=group1, square=10, **params)    
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False)    
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'boxes1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/boxes1.mp4' controls>alternative text</video>
<p></p>

A group can only have one box and each group can have its own box.

```python
group1 = dict(name='1', n=2, n_inf=1, box=[1,3,2,4])
group2 = dict(name='2', n=2, n_inf=0, box=[6,9,6,10])
groups = [group1, group2]

sim = Sim(groups=groups, square=10, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, 
    layout='dots_only', show_intro=False, use_init_func=False
)
chart.to_html5_video()
```

In [15]:
if RUN:
    group1 = dict(name='1', n=2, n_inf=1, box=[1,3,2,4])
    group2 = dict(name='2', n=2, n_inf=0, box=[6,9,6,10])
    groups = [group1, group2]
    
    sim = Sim(groups=groups, square=10, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False) 
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, 
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'boxes2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/boxes2.mp4' controls>alternative text</video>
<p></p>

Remember that a box only restricts the subjects in that group from *leaving* the space. Other dots not assigned to that box can move into the space without restriction.

```python
group1 = dict(name='1', n=5, n_inf=1, box=[1,3,1,3])
group2 = dict(name='2', n=5, n_inf=0)
groups = [group1, group2]

sim = Sim(groups=groups, square=10, **params)
sim.run()

chart = Chart(sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False)
chart.to_html5_video()
```

In [16]:
if RUN:
    params = {'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
    group1 = dict(name='1', n=5, n_inf=1, box=[1,3,1,3])
    group2 = dict(name='2', n=5, n_inf=0)
    groups = [group1, group2]

    sim = Sim(groups=groups, square=10, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False)
    HTML(chart.animate.to_html5_video())

if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, 
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'boxes3.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/boxes3.mp4' controls>alternative text</video>
<br>

___
#### Gates ####
Gates are the inverse of boxes. A gate is an area that subjects cannot enter.

Gates are a Gate object, which is a subclass of Grid, and they are created via the same 4 element iterable. For now, a gate can only be created by passing the `box_is_gated=True` flag as a keyword in a group dictionary.

Using the previous example, we can see that `group2` dots can no longer enter the `group1` box.

```python
group1 = dict(name='1', n=5, n_inf=1, box=[1,3,1,3], box_is_gated=True)
group2 = dict(name='2', n=5, n_inf=0)
groups = [group1, group2]

sim = Sim(groups=groups, square=10, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False
)
chart.animate.to_html5_video()
```

In [17]:
if RUN:
    group1 = dict(name='1', n=5, n_inf=1, box=[1,3,1,3], box_is_gated=True)
    group2 = dict(name='2', n=5, n_inf=0)
    groups = [group1, group2]

    sim = Sim(groups=groups, square=10, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, figsize=(16,8), dotsize=2000, layout='dots_only', show_intro=False, use_init_func=False
    )
    HTML(chart.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, 
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'gates1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/gates1.mp4' controls>alternative text</video>
<p></p>

This structure allows for intricate movement patterns. Here we show isolated groups:

```python
group1 = dict(name='1', n=50, n_inf=5, box=[1,5,1,20], box_is_gated=True)
group2 = dict(name='2', n=50, n_inf=5, box=[6,25,3,10], box_is_gated=True)
group3 = dict(name='3', n=50, n_inf=5, box=[10,21,16,22], box_is_gated=True)
group4 = dict(name='4', n=50, n_inf=5, box=[2,15,23,25], box_is_gated=True)
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, square=25, **params)
sim.run()

chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False, use_init_func=False)
chart.to_html5_video()
```

In [18]:
if RUN:
    group1 = dict(name='1', n=50, n_inf=5, box=[1,5,1,20], box_is_gated=True)
    group2 = dict(name='2', n=50, n_inf=5, box=[6,25,3,10], box_is_gated=True)
    group3 = dict(name='3', n=50, n_inf=5, box=[10,21,16,22], box_is_gated=True)
    group4 = dict(name='4', n=50, n_inf=5, box=[2,15,23,25], box_is_gated=True)
    groups = [group1, group2, group3, group4]

    sim = Sim(groups=groups, square=25, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), dotsize=200, layout='dots_only', show_intro=False)
    HTML(chart.animate.to_html5_video())

if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'gates2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/gates2.mp4' controls>alternative text</video>
<p></p>

And here some isolated and some free moving.

```python
group1 = dict(name='1', n=50, n_inf=5, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=50, n_inf=5, box=[14,19,14,19], box_is_gated=True)
group3 = dict(name='4', n=10, n_inf=5)
group4 = dict(name='4', n=10, n_inf=5)
groups = [group1, group2, group3, group4]

sim = Sim(groups=groups, square=25, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), layout='dots_only', 
      show_intro=False, use_init_func=False
)
chart.to_html5_video()
```

In [19]:
if RUN:
    group1 = dict(name='1', n=50, n_inf=5, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=50, n_inf=5, box=[14,19,14,19], box_is_gated=True)
    group3 = dict(name='4', n=10, n_inf=5)
    group4 = dict(name='4', n=10, n_inf=5)
    groups = [group1, group2, group3, group4]

    sim = Sim(groups=groups, square=25, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'gates3.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/gates3.mp4' controls>alternative text</video>
<br>

___
### Events ###
Events impact the attributes of subjects over the course of the simulation.

Events are utilized to better simulate real-world behavior. 

For instance, people do not move in consistent, prescribed ways. They move in regular ways most of the time with contacts that are well defined, but sometimes they attend events (perhaps periodically or uniquely) that are not governed by their regular movement patterns.

#### Event ####
An `Event` is an event that occurs at a particular location.

An `Event` accepts the following parameters:

* `xy`, the xy coordinates of the location
* `start_tick`, the tick when the event begins
* `groups`, an iterable of group ids that are eligible for the event
* `capacity`, the number of subjects that should attend
* `recurring`, how often the event recurs (i.e. every *nth* tick); if set to 0, the event does *not* recur

When an `Event` concludes, the subject returns to its `home` location as specified in the dot matrix.

To schedule an event, you must pass a list of event objects to the `events` parameter.

To begin with, we'll create a single `Event` object, called `show`, that occurs once on day 5.

```python
from rknot.events import Event

params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
group1 = dict(name='1', n=10, n_inf=5)

show = Event(name='show', xy=(5,5), start_tick=5, groups=[0], capacity=10)
events = [show]

sim = Sim(groups=group1, events=events, **params)
sim.run(dotlog=True)

chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)
chart.animate.to_html5_video()
```

In [20]:
if RUN:
    from rknot.events import Event

    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
    group1 = dict(name='1', n=10, n_inf=5)

    show = Event(name='show', xy=(5,5), start_tick=5, groups=[0], capacity=10)
    events = [show]

    sim = Sim(groups=group1, events=events, **params)
    sim.run(dotlog=True)

if SHOW_CHARTS:
    chart = Chart(sim, figsize=(16,8), layout='dots_only', show_intro=False)    
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'event1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/event1.mp4' controls>alternative text</video>
<p></p>

If you watch closely, you'll see on Day 5 that all the dots seemingly disappear, save for one, at location (5,5). 

In fact, all 10 dots are *actually at that location at the same time*.

We can confirm this by inspecting the [Dot Matrix](#) on that day via the `dotlog` attribute.

In [ ]:
from rknot.events import Event

params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}
group1 = dict(name='1', n=10, n_inf=5)

show = Event(name='show', xy=(5,5), start_tick=5, groups=[0], capacity=10)
events = [show]

sim = Sim(groups=group1, events=events, **params)
sim.run(dotlog=True)

In [ ]:
from rknot.dots import MATRIX_COL_LABELS as ML
sim.dotlog[4][:, ML['x']:ML['y']+1]

We can see the event more clearly if we extend the duration to 10 days. We also significantly reduced the frame rate.

```python
show = Event(name='show', xy=(5,5), start_tick=5, groups=[0], capacity=10, duration=10)
events = [show]

sim = Sim(groups=group1, events=events, **params)
sim.run()

chart = Chart(
    sim, figsize=(16,8), dotsize=1000, interval=300, 
    layout='dots_only', show_intro=False, use_init_func=False
)
chart.to_html5.video()
```

In [ ]:
if RUN:
    show = Event(name='show', xy=(5,5), start_tick=5, groups=[0], capacity=10, duration=10)
    events = [show]

    sim = Sim(groups=group1, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, figsize=(16,8), dotsize=1000, interval=300, 
        layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=300, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'event2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/event2.mp4' controls>alternative text</video>
<p></p>

Many event objects can be specified at once, in various combinations of groups.

```python
group1 = dict(name='1', n=10, n_inf=5)
group2 = dict(name='2', n=10, n_inf=0)

show = Event(name='show', xy=(5,5), start_tick=5, groups=[0,1], capacity=5, recurring=30)
game = Event(name='game', xy=(1,1), start_tick=5, groups=[0], capacity=5, recurring=14)
church = Event(name='church', xy=(1,1), start_tick=5, groups=[1], capacity=10, recurring=7)

groups = [group1, group2]
events = [show, game, church]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=300, dotsize=1000, layout='dots_only', show_intro=False,
    use_init_func=False
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    group1 = dict(name='1', n=10, n_inf=5)
    group2 = dict(name='2', n=10, n_inf=0)

    show = Event(name='show', xy=(5,5), start_tick=5, groups=[0,1], capacity=5, recurring=30)
    game = Event(name='game', xy=(1,1), start_tick=5, groups=[0], capacity=5, recurring=14)
    church = Event(name='church', xy=(1,1), start_tick=5, groups=[1], capacity=10, recurring=7)

    groups = [group1, group2]
    events = [show, game, church]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=300, dotsize=1000, layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'event3.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/event3.mp4' controls>alternative text</video>
<br>

___
#### Travel ####
Travel is a special type of event that allows a subject to enter a gate.

When a dot enters a gate via a Travel object, its box and gate attributes are temporarily adjusted to match those of the groups within the gate. The attributes revert when the event ends (determined by `duration` parameter).

Once inside the gate, the dot(s) are free to interact with other dots normally.

```python
from rknot.events import Travel

params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

visit = Travel(
    name='visit', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
)

groups = [group1, group2]
events = [visit]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False
    use_init_func=False
)
chartto_html5_video()
```

In [ ]:
if RUN:
    from rknot.events import Travel

    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

    group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

    visit = Travel(
        name='visit', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
    )

    groups = [group1, group2]
    events = [visit]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'travel1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/travel1.mp4' controls>alternative text</video>
<p></p>

Many unique layouturations can be achieved with this structure. Below, the `group1` box will be vacated by the solitary `group1` dot (essentially switching places with a dot from `group2`).

```python
from rknot.events import Travel

params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

visit2 = Travel(
    name='visit2', xy=[9,9], start_tick=3, groups=[0], capacity=1, duration=5, recurring=10
)
visit1 = Travel(
    name='visit1', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
)
groups = [group1, group2]
events = [visit2, visit1]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    from rknot.events import Travel

    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

    group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

    visit2 = Travel(
        name='visit2', xy=[9,9], start_tick=3, groups=[0], capacity=1, duration=5, recurring=10
    )
    visit1 = Travel(
        name='visit1', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
    )
    groups = [group1, group2]
    events = [visit2, visit1]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'travel2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/travel2.mp4' controls>alternative text</video>
<br>

___
#### Quarantine ###

A quarantine is an event object that makes several changes to a dots state
in order to restrict its movement.

When a dot is quarantined,
    
1. it goes back to its home location ([see Dot Matrix](#Dot-Matrix))
2. its boxes and gates are reset to match its group
3. its mover function is changed to 'quarantine'

In addition, a Quarantine object will create a additional restriction objects that disallow events during the quarantine ([see Restrictions below](#Restrictions))

```python
from rknot.events import Quarantine

params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

group1 = dict(name='1', n=2, n_inf=1, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=2, n_inf=0, box=[6,10,6,10], box_is_gated=True)

quar = Quarantine(name='all', start_tick=5, groups=[0,1], duration=30)

groups = [group1, group2]
events = [quar]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    from rknot.events import Quarantine

    params = {'square': 10, 'R0': 2.5, 'days': 50, 'imndur': 365, 'infdur': 365}

    group1 = dict(name='1', n=2, n_inf=1, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=2, n_inf=0, box=[6,10,6,10], box_is_gated=True)

    quar = Quarantine(name='all', start_tick=5, groups=[0,1], duration=30)

    groups = [group1, group2]
    events = [quar]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False,
        use_init_func=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'quar1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/quar1.mp4' controls>alternative text</video>

We can see from above that once in quaratine, the subjects barely move. We can include events in our structure. The events will be restricted during the quarantine period, then will resume when the quarantine ends.

```python
params = {'square': 10, 'R0': 2.5, 'days': 100, 'imndur': 365, 'infdur': 365}

group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

show = Event(name='show', xy=(6,6), start_tick=5, groups=[1], capacity=5, recurring=30)
visit2 = Travel(
    name='visit2', xy=[9,9], start_tick=3, groups=[0], capacity=1, duration=5, recurring=10
)
visit1 = Travel(
    name='visit1', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
)

groups = [group1, group2]
events = [show, visit2, visit1, quar]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=200, dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False,
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    params = {'square': 10, 'R0': 2.5, 'days': 100, 'imndur': 365, 'infdur': 365}

    group1 = dict(name='1', n=1, n_inf=1, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

    show = Event(name='show', xy=(6,6), start_tick=5, groups=[1], capacity=5, recurring=30)
    visit2 = Travel(
        name='visit2', xy=[9,9], start_tick=3, groups=[0], capacity=1, duration=5, recurring=10
    )
    visit1 = Travel(
        name='visit1', xy=[1,1], start_tick=3, groups=[1], capacity=1, duration=5, recurring=10
    )

    groups = [group1, group2]
    events = [show, visit2, visit1, quar]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=200, dotsize=2000, layout='no_details', show_intro=False
    )    
    HTML(chart.animate.to_html5_video())    
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=200, dotsize=5,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'quar2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/quar2.mp4' controls>alternative text</video>
<br>

___
#### Social Distancing ####
Event to mimick social distancing practices.

Social distancing is assumed to impact the Transmission Factor, $\tau$ of each dot. The core hypothesis is that practices such as maintaining 6-feet of distance or mask wearing don't reduce the number of contacts, but do reduce the likelihood that a contact will result in a new infection (ceterus paribus).

You can see use of this object [here](hit.ipynb#3.-Social-Distancing).

___
#### Vaccination ###
TBD

___
#### Restrictions ####
A restriction object restricts attendance to events that fall within the specified criteria. Each event has a `restricted` attribute that defaults to `False`. A restriction object filters out events from the event schedule by setting `restricted=True` for each event that satisfies the criteria.

To clarify, a Restriction is *not* an event. Events act on dots. Restrictions act on events.

Restrictions have potential as a versatile tool that can be used to investigate the impact of various government and business policy decisions that impact spread.

The Restriction object has a `criteria` parameter that accepts a `dict`, with keywords related to event object attributes. Acceptable `criteria` keys are currently:

In [ ]:
from rknot.helpers import md
from rknot.events import Restriction
msg = ' '.join(list(Restriction.OPERATORS.keys()))
md(msg)

The simplest way to restrict an event is by its name:

```python
from rknot.events import Restriction

params = {'square': 10, 'R0': 2.5, 'days': 100, 'imndur': 365, 'infdur': 365}

group1 = dict(name='1', n=10, n_inf=1, box=[1,5,1,5], box_is_gated=True)
group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

show1 = Event(name='show1', xy=(1,1), start_tick=2, groups=[0], capacity=10, recurring=2)
show2 = Event(name='show2', xy=(10,10), start_tick=2, groups=[1], capacity=10, recurring=2)


criteria = {'name': 'show1'}
res1 = Restriction(name='no_show1', start_tick=10, duration=20, criteria=criteria)

groups = [group1, group2]
events = [show1, show2, res1]

sim = Sim(groups=groups, events=events, **params)
sim.run(dotlog=True)

chart = Chart(
    sim, interval=300, dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False,
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    from rknot.events import Restriction

    params = {'square': 10, 'R0': 2.5, 'days': 100, 'imndur': 365, 'infdur': 365}

    group1 = dict(name='1', n=10, n_inf=1, box=[1,5,1,5], box_is_gated=True)
    group2 = dict(name='2', n=10, n_inf=0, box=[6,10,6,10], box_is_gated=True)

    show1 = Event(name='show1', xy=(1,1), start_tick=2, groups=[0], capacity=10, recurring=2)
    show2 = Event(name='show2', xy=(10,10), start_tick=2, groups=[1], capacity=10, recurring=2)


    criteria = {'name': 'show1'}
    res1 = Restriction(name='no_show1', start_tick=10, duration=20, criteria=criteria)

    groups = [group1, group2]
    events = [show1, show2, res1]

    sim = Sim(groups=groups, events=events, **params)
    sim.run(dotlog=True)
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=300, dotsize=2000, layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())      
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=300,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'restrict1.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/restrict1.mp4' controls>alternative text</video>
<p></p>

In the above, we can see that every other day both group boxes have events that are attended by all dots in the group.

But on day 10, the `group1` dots no longer converge on location (1,1). Instead, they are spread throughout their box. So `res1` has successfully restricted attendance to `show1`.

Unlike `Quarantine` objects, however, the `group1` dots have not changed their standard movement patterns.

We can restrict multiple events via the other criteria. Next we will restrict events based on their capacity. Events with more than 5 subjects in attendance will be restricted.

```python
criteria = {'capacity': 5}
res1 = Restriction(name='cap5', start_tick=10, duration=20, criteria=criteria)

groups = [group1, group2]
events = [show1, show2, res1]

sim = Sim(groups=groups, events=events, **params)
sim.run()

chart = Chart(
    sim, interval=300, dotsize=2000, layout='dots_only', show_intro=False,
    use_init_func=False,
)
chart.to_html5_video()
```

In [ ]:
if RUN:
    criteria = {'capacity': 5}
    res1 = Restriction(name='cap5', start_tick=10, duration=20, criteria=criteria)

    groups = [group1, group2]
    events = [show1, show2, res1]

    sim = Sim(groups=groups, events=events, **params)
    sim.run()
if SHOW_CHARTS:
    chart = Chart(
        sim, interval=300, dotsize=2000, layout='dots_only', show_intro=False
    )
    HTML(chart.animate.to_html5_video())
if SAVE_CHARTS:
    chart_params = dict(
            show_intro=False, use_init_func=False, 
            interval=300,
            layout='dots_only', reso_format='web'
        )
    chart = Chart(sim, **chart_params).save(PATH + 'restrict2.mp4')

<video width='705' height='340' 
    src='https://storage.googleapis.com/rknotvids/keycons/restrict2.mp4' controls>alternative text</video>
<p></p>

In the above we see that neither of the groups had events from day 10 onward during the restriction period.

Restrictions can be chained together as desired to form a complex and tailored policy recipe for the population of the sim. [See this example](#).

### Dot Matrix ###

In [ ]:
from rknot.dots import MATRIX_LABELS as LABELS
msg = "The dot matrix is essentially RKnot's canonical form of data structure. "
msg += "The matrix is simply a 2D `numpy` array of shape (*n*, {}) ".format(len(LABELS))
msg += "with each of the *n* rows representing a dot and each column representing "
msg += "an attribute.\n"

md(msg)

 More typical Python objects have been eschewed in favor the Dot Matrix because:

* **RKnot** relies heavily on [Ray](https://docs.ray.io/en/latest/) for parallel processing and [Numba](https://numba.pydata.org/) for just-in-time compilation and vectorization to improve processing speed.
* `Numpy` arrays have several advantages in **Ray** including [rapid serialization](https://ray-project.github.io/2017/10/15/fast-python-serialization-with-ray-and-arrow.html) and ease of batching.
* **Numba** also integrates well with `numpy`, supporting many of its features and leads to major performance improvements.

The dot matrix is created inside a **Ray** actor at instantiation and is only passed back to the main `Sim` object when the simulation is completed.

It can be accessed via the `dots` attribute. Below is a sample of 4 dots:

In [ ]:
sim.dots[:4]

The column attributes have corresponding labels:

In [ ]:
from rknot.dots import MATRIX_LABELS
print (MATRIX_LABELS)

With these labels, the 4 dot matrix above can be shown in a table.

In [ ]:
msg = '| ' + ' '.join(['{} |'.format(label) for label in MATRIX_LABELS])
msg += ' \n'
msg += '| ' + ' '.join([':---: |' for label in MATRIX_LABELS])
for dot in sim.dots[:5]:
    msg += ' \n'
    msg += '| ' + ' '.join(['{} |'.format(dot[i]) for i, label in enumerate(MATRIX_LABELS)])
    
md(msg)

There are several data types at work:    

* *categorical integers*; used to identify related objects
    * `id`, `group_id`, `loc_id`, `home_id`, `box_id`, `event_id`, `mover`
    
* *boolean integers*; used to set boolean flags
    * `0` means `False` and `1` means `True`
    * `is_alive`, `is_vaxxed`, `is_sus`, `is_inf`, `ever_inf`, `go_home`

* *coordinates*; used to identify locations
    * `x`, `y`, `homex`, `homey`

* *event ticks*; integers that trigger an event on the given tick
    * `depart`, `recover`, `relapse`

* *factors*; scaled integers that must be unscaled before being used in multiplicative formulas
    * `tmf`, `ifr`

The column attributes are defined as follows:

In [ ]:
from rknot.dots.matrix import MATRIX_COL_DEFNS as DEFNS
table = '| Label | Definition '*2 + '|'
table += ' \n'
table += '|:---:|:---'*2 + '|'
table += ' \n'
defns = list(DEFNS.items())
tablen = len(defns) // 2
for i,j in zip(defns[:tablen], defns[tablen:]):
    table += '| ' + i[0] + ' | ' + i[1] + ' |' + j[0] + ' | ' + j[1] + ' |'
    table += '\n'
    
md(table)

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
from rknot.helpers import move_to_doc_folder
move_to_doc_folder('concepts.ipynb')